In [23]:
from mesa import Model
from mesa import Agent
from mesa.space import MultiGrid
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector
import random
import numpy as np
import pickle
import cv2
from IPython.display import clear_output

from a_star_pathfinding import a_star
from vision import lines
from vision import transform

In [24]:
# make seperate function for imposter
def imposter_strategy(pos, strategy, tasks, kill):
    
    if strategy == 'hunter':
        goal = [random.choice(tasks).pos]
    
    if strategy == 'simp':
        if kill:
            goal = [random.choice(tasks).pos]
        else:
            goal = []
            
    return goal

In [25]:
def euclidean(pos_a, pos_b):
    return ((pos_a[0] - pos_b[0])**2 + (pos_a[1] - pos_b[1])**2)**.5

In [26]:
class AmongUs(Model):
    def __init__(self, map_name, n_crew, n_impo):
        super().__init__()

        # generate grid
        self.height = 138
        self.width = 242
        self.grid = MultiGrid(self.width, self.height, torus=True)

        # generate map
        self.generate_map(map_name)
        
        # generate tasks
        self.generate_tasks(map_name)
        
        # load vision dict for imposters
        self.vision_dict = np.load('vision_dict.pkl', allow_pickle=True)
        
        # create schedulers for agents
        self.schedule_Crewmate = RandomActivation(self)
        self.schedule_Imposter = RandomActivation(self)
        
        # initialize agents
        self.n_crew = n_crew
        self.n_impo = n_impo
        self.spawn_players()
        
        #count the run number
        self.run = 0
        
    def generate_map(self, map_name):
        hard_walls = np.load(f'{map_name}/hardwalls.npy')
        vents = np.load(f'{map_name}/vents.npy', allow_pickle=True)
        obstructions = np.load(f'{map_name}/obstructions.npy', allow_pickle=True)
        
        for coord in hard_walls:
            self.new_agent(Wall, tuple(coord))

        self.vents_dict = {}
        self.vents = []

        for connection in vents:
            connection = connection.tolist()
            self.vents_dict[tuple(connection[0])] = tuple(connection[1])

        for coord in self.vents_dict:
            self.new_agent(Vent, tuple(coord))
        
        for coord in obstructions:
            self.new_agent(Obstruction, tuple(coord))

    def generate_tasks(self, map_name):
        # load the image + coordinates of the short tasks
        short_tasks = np.load(f'{map_name}/shorttasks.npy', allow_pickle=True)
        self.short_tasks = short_tasks
        self.tasks = []

        # load all the short_tasks into the map (all possible task locations), first create tuples
        for i in range(len(short_tasks)):
            if i == 3:
                self.new_agent(ShortTask, tuple(short_tasks[i][0][0]))
                for parttwo in range(8):
                    self.new_agent(ShortTask, tuple(short_tasks[i][1][parttwo]))
            elif i == 6:
                for partone in range(5):
                    self.new_agent(ShortTask, tuple(short_tasks[i][0][partone]))
                self.new_agent(ShortTask, tuple(short_tasks[i][1][0]))
        
            else:
                self.new_agent(ShortTask, tuple(short_tasks[i][0]))

    def social_matrixes(self, self.run):
        #trust matrix does not change throughout a game
        self.trust_matrix = np.load(f'social_matrixes/trust_{run}.npy')
        
        #sus matrix changes throughout game
        self.sus_matrix = np.load(f'social_matrixes/sus_{run}.npy')
    
    def spawn_players(self):
        # generate crewmates
        for _ in range(self.n_crew):
            self.new_agent(Crewmate, (125, 100))

        # generate imposters
        for _ in range(self.n_impo):
            self.new_agent(Imposter, (125, 100))

    def new_agent(self, agent_type, pos):
        '''
        Method that creates a new agent, and adds it to the correct scheduler.
        '''
        agent = agent_type(self.next_id(), self, pos)

        self.grid.place_agent(agent, pos)

        if agent_type == Crewmate or agent_type == Imposter:
            getattr(self, f'schedule_{agent_type.__name__}').add(agent)

        if agent_type == Vent:
            self.vents.append(agent)
        
        if agent_type == ShortTask:
            self.tasks.append(agent)

    def step(self):
        '''
        Method that steps every agent. 
        
        Prevents applying step on new agents by creating a local list.
        '''

        self.schedule_Crewmate.step()
        self.schedule_Imposter.step()

        # self.datacollector.collect(self)
        
        #update sus matrix every step (diagonal should be changed back to 0 at the end)
        self.sus_matrix += 0.005

    def play_match(self, step_count=200):
        '''
        Method that runs the model for a specific amount of steps.
        '''
        for i in range(step_count):
            print(f'Current step: {i}')
            self.step()
            # clear_output(wait=True)


In [27]:
class Crewmate(Agent):
    def __init__(self, unique_id, model, init_pos):
        super().__init__(unique_id, model)
        self.pos = init_pos
        self.path = []
        self.goallist = []
        self.injob = 0

        # self.create_dict()
        
    def make_tasks(self):
        short_tasks = self.model.short_tasks

        # AANPASSING
        # set 4 random tasks for Crewmate to execute (can still be same task multiple times)
        self.goallist = random.sample(list(short_tasks), 4)
        
        # if it is a task with multiple locations/second part, replace by one of the coordinates from the first list 
        self.goallist = [[random.choice(self.goallist[index][0])] if len(x)>1 else x for index,x in enumerate(self.goallist)]
        
        # Turn list into list of tuples ### GEEFT SOMS EEN ERROR 
        self.goallist = [tuple(self.goallist[index][0]) for index,y in enumerate(self.goallist)]
        
    def find_path(self):
        if len(self.goallist) == 0:
            self.make_tasks()
        
        possiblepaths = []
        for t in self.goallist:
            possiblepaths.append(a_star(self.pos, t, self.model.grid)[0])
            
        self.path = self.path + min(possiblepaths)

    def step(self):
        if self.path == []:
            if self.pos in self.goallist:
                self.goallist.remove(self.pos)
                self.injob = random.randint(5, 15)
                
            # if Task was task 3, add second part of task 3 to the task list
            if self.pos == tuple(self.model.short_tasks[3][0][0]):
                self.goallist.append(tuple(random.choice(self.model.short_tasks[3][1])))
            
            # if Task was task 6, add second part of task 6 to the task list
            elif self.pos in self.model.short_tasks[6][0]:
                self.goallist.append(tuple(self.model.short_tasks[6][1][0]))
            
            self.find_path()

        elif self.path != []:
            if self.injob > 0:
                self.injob -= 1
            else:
                self.model.grid.move_agent(self, self.path.pop())    
    
    def die(self):
        self.new_agent(Dead_crewmate, self.pos)
        self.model.remove_agent(self)


#########################################################################################################
    def create_dict(self):
        vision_dict = {}
        grid = self.model.grid
        for i in range(242):
            for j in range(138):     
                print(f'Current row: {i}/242') 
                print(f'Current col: {j}/138')
                print(f'Current position: {self.pos}')
                is_wall = False
                agents_on_tile = grid.get_neighbors(pos=(i,j), moore=True, radius=0, include_center=True)

                # stop if wall
                for agent in agents_on_tile:
                    if type(agent) == Wall:
                        is_wall = True

                if is_wall == False:
                    self.pos = (i,j)
                    vision_neighborhood = self.find_vision()
                    vision_dict[(i,j)] = vision_neighborhood
                
                clear_output(wait=True)        

        a_file = open("vision_dict2.pkl", "wb")
        pickle.dump(vision_dict, a_file)
        a_file.close()

    def find_vision(self):
        '''
        Determines the vision of the agent 
        '''
        grid = self.model.grid
        origin_lines = lines
        transformed_lines = transform(origin_lines, self.pos)
        vision_neighborhood = []
        
        for i in range(len(transformed_lines)):
            found_wall = False
            for j in range(len(transformed_lines[i])):  
                agents_on_tile = grid.get_neighbors(pos=transformed_lines[i][j], moore=True, radius=0, include_center=True)

                # stop line if it bumps into a wall  
                for agent in agents_on_tile:
                    if type(agent) == Wall:
                        found_wall = True
                
                # append to vision neigborhood when no wall is found and if coordinate is not already appended before
                if found_wall == False:
                    if transformed_lines[i][j] not in vision_neighborhood:
                        vision_neighborhood.append(transformed_lines[i][j])  
                        
        return vision_neighborhood

#########################################################################################################
        
            
class Imposter(Agent):
    def __init__(self, unique_id, model, init_pos):
        super().__init__(unique_id, model)
        self.pos = init_pos
        self.path = []
        self.goals = [(85, 35), (140, 110), (125, 90)]
        self.vents_enabled = True
        self.tactic = 'simp'

    def make_task(self, tactic, tasks):
        self.goals = imposter_strategy(self.pos, tactic, tasks, False)
    
    def find_path(self):
        if len(self.goals) == 0:
            self.make_task(self.tactic, self.model.tasks)
        
        else:
            goal = self.goals.pop()
            best_path, best_score = a_star(self.pos, goal, self.model.grid)

            if self.vents_enabled:
                for vent in self.model.vents:
                    vent_in = vent.pos_a
                    vent_out = vent.pos_b
                    guess_vent_path = euclidean(self.pos, vent_in) + euclidean(vent_out, goal)

                    if guess_vent_path < best_score:
                        vent_path_in, vent_score_in = a_star(self.pos, vent_in, self.model.grid)
                        vent_path_out, vent_score_out = a_star(vent_out, goal, self.model.grid)
                        vent_path = vent_path_out + vent_path_in
                        vent_score = vent_score_in + vent_score_out
    
                        if vent_score < best_score:
                            best_score = vent_score
                            best_path = vent_path

            self.path = best_path

    def detect_crewmates(self):
        crewmates_detected = []
        visible_area = self.model.vision_dict[self.pos]
        for pos in visible_area:
            agents_on_tile = self.model.grid.get_neighbors(pos=pos, moore=True, radius=0, include_center=True)
            for agent in agents_on_tile:
                if type(agent) == Crewmate:
                    crewmates_detected.append(agent)
        return crewmates_detected

    def step(self):
        if self.path == []:
            self.find_path()

        crewmates_detected = self.detect_crewmates()

        for mate in crewmates_detected:
            print(mate.pos)

        if self.path != []:
            self.model.grid.move_agent(self, self.path.pop())  
            
class Dead_crewmate(Agent):
    def __init__(self, unique_id, model, init_pos):
        super().__init__(unique_id, model)
        self.pos = pos    
    
class Wall(Agent):
    def __init__(self, unique_id, model, pos):
        super().__init__(unique_id, model)
        self.pos = pos

class Obstruction(Agent):
    def __init__(self, unique_id, model, pos):
        super().__init__(unique_id, model)
        self.pos = pos

class Vent(Agent):
    def __init__(self, unique_id, model, pos):
        super().__init__(unique_id, model)
        self.pos_a = pos
        self.pos_b = self.model.vents_dict[pos]

class ShortTask(Agent):
    def __init__(self, unique_id, model, pos):
        super().__init__(unique_id, model)
        self.pos = pos

In [28]:
amongUs = AmongUs('the_skeld', 3, 1)

amongUs.play_match()

Current step: 0
(125, 100)
(125, 100)
(125, 100)
Current step: 1
(126, 99)
(126, 99)
(124, 101)
Current step: 2
(127, 98)
(127, 98)
(123, 102)
Current step: 3
(128, 97)
(128, 97)
(122, 103)
Current step: 4
(129, 96)
(129, 96)
(121, 104)
Current step: 5
(130, 95)
(130, 95)
(120, 105)
Current step: 6
(131, 94)
(131, 94)
(119, 105)
Current step: 7
(132, 93)
(132, 93)
(118, 105)
Current step: 8
(133, 92)
(133, 92)
(117, 105)
Current step: 9
(134, 91)
(134, 91)
(116, 105)
Current step: 10
(134, 90)
(134, 90)
(115, 105)
Current step: 11
(114, 105)
(134, 89)
(134, 89)
Current step: 12
(113, 105)
(134, 88)
(134, 88)
Current step: 13
(112, 105)
(134, 87)
(134, 87)
Current step: 14
(134, 86)
(134, 86)
Current step: 15
(134, 85)
(134, 85)
Current step: 16
(134, 84)
(134, 84)
Current step: 17
(134, 83)
(134, 83)
Current step: 18
(134, 82)
(134, 82)
Current step: 19
(134, 81)
(134, 81)
Current step: 20
(134, 80)
(134, 80)
Current step: 21
Current step: 22
Current step: 23
Current step: 24
Current s

AttributeError: 'Imposter' object has no attribute 'tactic'